In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
from qubex.pulse import FlatTop, PulseSequence
from qubex.simulator import Control, Coupling, Simulator, System, Transmon

In [ ]:
transmons = [
    Transmon(
        label="Q08",
        dimension=3,
        frequency=7.646,
        anharmonicity=-0.356,
        decay_rate=0.00005,
        dephasing_rate=0.00005,
    ),
    Transmon(
        label="Q09",
        dimension=3,
        frequency=8.442,
        anharmonicity=-0.448,
        decay_rate=0.00005,
        dephasing_rate=0.00005,
    ),
    Transmon(
        label="Q10",
        dimension=3,
        frequency=8.316,
        anharmonicity=-0.412,
        decay_rate=0.00005,
        dephasing_rate=0.00005,
    ),
    Transmon(
        label="Q11",
        dimension=3,
        frequency=7.192,
        anharmonicity=-0.368,
        decay_rate=0.00005,
        dephasing_rate=0.00005,
    ),
]

couplings = [
    Coupling(pair=("Q08", "Q09"), strength=0.01),
    Coupling(pair=("Q10", "Q11"), strength=0.01),
    Coupling(pair=("Q08", "Q10"), strength=0.01),
    Coupling(pair=("Q09", "Q11"), strength=0.01),
]

system = System(
    transmons=transmons,
    couplings=couplings,
)

In [ ]:
system.draw()

In [ ]:
system.hamiltonian

In [ ]:
simulator = Simulator(system)

In [ ]:
initial_state = system.state(
    {
        "Q08": "0",  # |0>
        "Q09": "-i",  # |-i>
        "Q10": "+",  # |+>
        "Q11": "*",  # random state in qubit subspace
    },
)

In [ ]:
duration = 30
tau = 10
pi_x = FlatTop(
    duration=duration,
    amplitude=np.pi / (duration - tau),
    tau=tau,
)
pi_y = pi_x.shifted(np.pi / 2)
pulse = PulseSequence([pi_x, pi_y])
pulse.plot()
pulse.values

In [ ]:
target = transmons[0]

detuning = 0.001

control = Control(
    target=target.label,
    frequency=target.frequency + detuning,
    waveform=pulse.values,
)
control.plot()
control.plot(polar=True)

In [ ]:
result = simulator.simulate(
    control=control,
    initial_state=initial_state,
)

In [ ]:
final_state = result.states[-1]
final_state

In [ ]:
final_state_target = result.substates("Q08")[-1]
final_state_target

In [ ]:
result.plot_population_dynamics("Q08")

In [ ]:
result.show_last_population("Q08")

In [ ]:
result.display_bloch_sphere("Q08", frame="drive")
result.display_bloch_sphere("Q08", frame="qubit")

In [ ]:
result.display_bloch_sphere("Q09", frame="qubit")
result.display_bloch_sphere("Q10", frame="qubit")
result.display_bloch_sphere("Q11", frame="qubit")